In [3]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.config import get_paths
from src.IO import read_parquet

paths = get_paths(project_root)

In [7]:
orders = read_parquet(paths.root / "data" / "interim" / "orders_final.parquet")
customers = read_parquet(paths.root / "data" / "interim" / "customers_final.parquet")
order_items = read_parquet(paths.root / "data" / "interim" / "order_items_final.parquet")
payments = read_parquet(paths.root / "data" / "interim" / "order_payments_final.parquet")

In [19]:
order_revenue = order_items.groupby("order_id").agg(revenue=("price", "sum"),freight_value=("freight_value", "sum"),n_of_items=("order_item_id", "count")).reset_index()

In [17]:
order_revenue.head()

,revenue,freight_value,n_of_items,total
order_id,,,,
00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,1,72.19
00018f77f2f0320c557190d7a144bdd3,239.90,19.93,1,259.83
000229ec398224ef6ca0657da4fc703e,199.00,17.87,1,216.87
00024acbcdf0a6daa1e931b038114c75,12.99,12.79,1,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,1,218.04


In [13]:
payment_agg = payments.groupby("order_id").agg(total_paid=("payment_value", "sum"),n_of_payments=("payment_type", "count"))

In [14]:
payment_agg.head()


,total_paid,n_of_payments
order_id,,
00010242fe8c5a6d1ba2dd792cb16214,72.19,1
00018f77f2f0320c557190d7a144bdd3,259.83,1
000229ec398224ef6ca0657da4fc703e,216.87,1
00024acbcdf0a6daa1e931b038114c75,25.78,1
00042b26cf59d7ce69dfabb4e55b4fd9,218.04,1


In [22]:
order_check = (
    order_revenue
    .merge(payment_agg, on="order_id", how="inner")
)

In [24]:
order_check[order_check["order_id"] =="00042b26cf59d7ce69dfabb4e55b4fd9" ]

,order_id,revenue,freight_value,n_of_items,total_paid,n_of_payments
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.9,18.14,1,218.04,1


An order-level check table was created to inspect random samples and verify that item-
and payment-level aggregations were performed correctly. Minor differences between
item totals and payment values are expected due to discounts, vouchers, and split
payments. Aggregated payment values are used as the source of transactional revenue
in downstream analysis.


In [26]:
orders_fact = (
    orders
    .merge(order_revenue, on="order_id", how="left")
    .merge(payment_agg, on="order_id", how="left")
)


In [27]:
orders_fact.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,revenue,freight_value,n_of_items,total_paid,n_of_payments
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,29.99,8.72,1.0,38.71,3.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,118.70,22.76,1.0,141.46,1.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,159.90,19.22,1.0,179.12,1.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,45.00,27.20,1.0,72.20,1.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,19.90,8.72,1.0,28.62,1.0


In [29]:
orders_fact.shape

(97585, 11)

In [31]:
customers.head()

,customer_id,customer_unique_id,customer_state,customer_city
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,SP,franca
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,SP,sao bernardo do campo
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,SP,sao paulo
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,SP,mogi das cruzes
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,SP,campinas


In [32]:
orders_customers = orders_fact.merge(
    customers[["customer_id", "customer_unique_id", "customer_state"]],
    on="customer_id",
    how="left"
)

In [33]:
orders_customers.shape

(97585, 13)

In [34]:
orders_customers.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,revenue,freight_value,n_of_items,total_paid,n_of_payments,customer_unique_id,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,29.99,8.72,1.0,38.71,3.0,7c396fd4830fd04220f754e42b4e5bff,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,118.70,22.76,1.0,141.46,1.0,af07308b275d755c9edb36a90c618231,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,159.90,19.22,1.0,179.12,1.0,3a653a41f6f9fc3d2a113cf8398680e8,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,45.00,27.20,1.0,72.20,1.0,7c142cf63193a1473d2e66489a9ae977,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,19.90,8.72,1.0,28.62,1.0,72632f0f9dd73dfee390c9b22eb56dd6,SP


In [39]:
customer_metrics = (
    orders_customers
    .groupby("customer_unique_id", as_index=False)
    .agg(
        first_purchase_date=("order_purchase_timestamp", "min"),
        last_purchase_date=("order_purchase_timestamp", "max"),
        n_orders=("order_id", "nunique"),
        total_revenue=("total_paid", "sum"),
        avg_order_value=("total_paid", "mean"),
        customer_state=("customer_state", "first")
    )
)

In [44]:
orders_customers.groupby("customer_unique_id")["customer_state"].nunique().value_counts()

customer_state
1    94362
2       36
3        1
Name: count, dtype: int64

In [43]:
customer_metrics.head()

,customer_unique_id,first_purchase_date,last_purchase_date,n_orders,total_revenue,avg_order_value,customer_state
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,2018-05-10 10:56:27,1,141.90,141.90,SP
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,2018-05-07 11:11:27,1,27.19,27.19,SP
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,2017-03-10 21:05:03,1,86.22,86.22,SC
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,2017-10-12 20:29:41,1,43.62,43.62,PA
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,2017-11-14 19:45:42,1,196.89,196.89,SP


In [45]:
customer_metrics["tenure_days"] = (
    customer_metrics["last_purchase_date"]
    - customer_metrics["first_purchase_date"]
).dt.days

In [46]:
customer_metrics.head()

,customer_unique_id,first_purchase_date,last_purchase_date,n_orders,total_revenue,avg_order_value,customer_state,tenure_days
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,2018-05-10 10:56:27,1,141.90,141.90,SP,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,2018-05-07 11:11:27,1,27.19,27.19,SP,0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,2017-03-10 21:05:03,1,86.22,86.22,SC,0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,2017-10-12 20:29:41,1,43.62,43.62,PA,0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,2017-11-14 19:45:42,1,196.89,196.89,SP,0


In [48]:
customer_metrics.shape

(94399, 8)

In [49]:
customer_metrics.isna().sum()

customer_unique_id     0
first_purchase_date    0
last_purchase_date     0
n_orders               0
total_revenue          0
avg_order_value        1
customer_state         0
tenure_days            0
dtype: int64

In [50]:
customer_metrics.duplicated().sum()

np.int64(0)

In [52]:
customer_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94399 entries, 0 to 94398
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   customer_unique_id   94399 non-null  object        
 1   first_purchase_date  94399 non-null  datetime64[ns]
 2   last_purchase_date   94399 non-null  datetime64[ns]
 3   n_orders             94399 non-null  int64         
 4   total_revenue        94399 non-null  float64       
 5   avg_order_value      94398 non-null  float64       
 6   customer_state       94399 non-null  object        
 7   tenure_days          94399 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(2), object(2)
memory usage: 5.8+ MB


In [51]:
customer_metrics.describe()

,first_purchase_date,last_purchase_date,n_orders,total_revenue,avg_order_value,tenure_days
count,94399,94399,94399.000000,94399.000000,94398.000000,94399.000000
mean,2018-01-01 05:09:43.121600768,2018-01-03 20:55:27.464761344,1.033750,165.252553,160.344348,2.645558
min,2016-09-04 21:15:19,2016-09-04 21:15:19,1.000000,0.000000,9.590000,0.000000
25%,2017-09-13 08:37:13.500000,2017-09-17 00:11:51.500000,1.000000,63.100000,62.390000,0.000000
50%,2018-01-19 21:09:30,2018-01-22 22:33:20,1.000000,107.780000,105.600000,0.000000
75%,2018-05-04 22:53:05.500000,2018-05-07 13:53:04,1.000000,182.610000,176.650000,0.000000
max,2018-08-29 15:00:37,2018-09-03 09:06:57,16.000000,13664.080000,13664.080000,633.000000
std,NaN,NaN,0.210542,226.136236,219.423639,25.020976


In [47]:
from src.IO import write_parquet

write_parquet(
    customer_metrics,
    paths.root / "data" / "processed" / "customer_metrics.parquet"
)